асинкио - это короче чуть улушенная тема по сравнению с тредингом, но и она не решает весь спектр проблем, поэтому иногда приходится таки использовать трендинг

у асинкио есть спец слово - async, оно ставится перед def при определениии функции,
сами ф-и построены на генераторах

генераторы возвращают ленивые итераторы - ленивые генераторы, которые через некст вроде как работают
функции корутины возвращают объекты корутины

корутины позволяют строить цепочки неблокирующих вызовов. Блокирующий вызов является кодом, который не дает CPU делать что-либо еще в течение некоторого периода времени, 
в питон 3.5 изменился синтаксис корутинов, при этом их действия не изменилось.

async - прописывает перед ф-ей, сделать ее корутином, ставится перед def
___
await - прописывается перед методом, он же - ф-я, он используется для асинхронного ожидания результата от выполнения корутина,
то есть это асинхронный вызов, так вот, все, что после эуэйт идет, будет исполнено после того, как завершит работу сам эуэйт. Эуэйт делается на эуэйтбл объекте, в примере ниже этот эуэйтбл объект возвращается после вызова ф-и gather 


программа, использующая корутины, должна запустить event loop, цикл обработки событий
его можно запустить из любой точки синхронной части программы, из любой синхронной ф-и вызвать асинхронную

eventloop запускается вызовом asyncio.run()

ивентлуп откроется, а когда вся асинхронка закончится, ивентлуп закроется

ИЛ отвечает за запуск корутинов, колбэков, финализацию асинхронных генераторов и тд

под разными ОС запускаются разные реализации ИЛ: для винды ProactorEventLoop, SelectorEventLoop для линукс

есть несколько реализаций вообще асинронных модулей, одна из них uvloop, мб когда нибудь ее и изучу

доступен API для ручного управления ИЛ


когда мы вызываем корутины, мы перед ними ставим await для того, чтобы вызов произошел асинхоронно, и код, который находится после await, продолжил исполняться после того, как придет результат из вызова, ну или завершится этот асинхронный вызов

как только он завершится, исполнение продолжится, допустим, со следующей строчки кода



для того, чтобы асинх ф-я вызвала другую ф-ю, вызываемая ф-я должна быть тоже асинх, кроме того, методы, используемые во второй ф-и также дб асинх

In [6]:
# в этом примере запускается через await main(), хотя надо запустить ивентлуп через asyncio.run(main())
import asyncio

async def tick():  # вызываемая ф-я асинк
    print('tick')
    await asyncio.sleep(1)  # у них и слип есть свой асинх, 
    print('tack')
    
async def main(): # основная ф-я тоже асинк
    await asyncio.gather(tick(), tick(), tick())  # метод всегда с await, gather - типа map
    
    
if __name__ == '__main__':
    await main()

ModuleNotFoundError: No module named 'multythreading'

получается типа у нас есть три потока, пришли в ф-ю тик, запринтили первый принт ТИК, поспали секунду, запринтили ТАК, 

In [ ]:
import asyncio

from multythreading.decorators import async_measure_time


async def tick():  # вызываемая ф-я асинк
    print('tick')
    await asyncio.sleep(1)  # у них и слип есть свой асинх,
    print('tack')


@async_measure_time  # тут мы сделали декоратор
async def main():  # основная ф-я тоже асинк
    await asyncio.gather(tick(), tick(), tick())  # метод всегда с await, gather - типа map


if __name__ == '__main__':
    asyncio.run(main())


какая тема: мы прописали декоратор @measure_time, навели на него и через контроль+альт открыли в пайчарме файлик с декоратором, 
переделали его в асинх, он стал таким:
import time
from functools import wraps


def async_measure_time(func):

    @wraps(func)
    async def wrap(*args, **kwargs):
        start = time.perf_counter()
        result = await func(*args, **kwargs)
        finish = time.perf_counter() - start
        print(f'it took {finish:0.2f} to execute {func}')
        return result
    return wrap

то есть немного изменили имя самой ф-и, потом добавили асинк к враперу и эуэйт к ф-и в переменной результат, потом начали использовать
в юпитере это не сработало

первые итоги:

    1. код исполняется за одну реальную секунду, то есть когда речь идет об инпут-аут операциях, это работает почти реально параллельно и работает за одну секунду.
    2. такую же тему можно было бы сделать и использую трединг, однако код был бы уже сложнее, а здесь вся основная писанина и работа с потоками скрыта от нас, хотя основные моменты мы явно декларируем, пишем асинк и эуэйт, чтобы и машина понимала, что мы от нее хотим. Но прописывать две эти строчки явно проще, чем манипулировать тредами
    3. вызовы происходят асинхронно, и уэуйт возвращает на время выполнения IO-bound операции управление вызывающему коду. То есть это не блокирующие вызовы
    4. эуэйт мб использован только внутри корутина, мы не можем не из корутина вызвать корутин при помощи эуэйт. Т. к. использование асинк/эуэйт пронизывает весь стек вызовов
    5. мы в корутине можем вызвать некорутин, но не наоборот
    6. эуэйт обязан быть прописанным при вызове корутина
    7. yeild развешен в корутинах и создает асинх генератор
    8. return также разрешен
    9. эуэйт может быть использован не только при вызове корутина, но и при вызове эуэйтбл объекта
    10. эуэйтбл - объект, реализующий дандер метод __awaitable__(), который возвращает итератор
    11. корутин относится к эуэйтбл объектам
    

In [ ]:
import asyncio




async def tick():  # вызываемая ф-я асинк
    print('tick')
    await asyncio.sleep(1)  # у них и слип есть свой асинх,
    print('tack')
    loop2 = asyncio.get_running_loop()
    if loop2.is_running():
        print('loop is running')

async def main():  # основная ф-я тоже асинк
    await asyncio.gather(tick(), tick(), tick())  # метод всегда с await, gather - типа map


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(main())  # блокирующий вызов
        print('coroutines have finished')

    finally:
        loop.close()
        print('loop has been closed')



In [ ]:
# этот пример, если мы хотим сделать так, чтобы после завершения ф-и поток продолжал работу, 
# для этого в эдин конфигурейшн надо поставить эмулятор терминала2
import asyncio




async def tick():  # вызываемая ф-я асинк
    print('tick')
    await asyncio.sleep(1)  # у них и слип есть свой асинх,
    print('tack')
    loop2 = asyncio.get_running_loop()
    if loop2.is_running():
        print('loop is running')

async def main():  # основная ф-я тоже асинк
    await asyncio.gather(tick(), tick(), tick())  # метод всегда с await, gather - типа map


if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    try:
        loop.create_task(mfain())  # блокирующий вызов
        loop.run_forever()
        print('coroutines have finished')
    except KeyboardInterrupt:
        print('app was closed manually')
    finally:
        loop.close()
        print('loop has been closed')

